# Heatmaps and Networks

* We can use the method to plot all patterns, double click on specific patterns to highlight them in the heatmaps, and select top patterns to plot them on their own.

* A html version of the notebook is available at https://trangdang168.github.io/CRIM_Intervals_Notebooks/Trang_CRIM_Intervals_Morgan.html. If this notebook doesn't work properly, please visit this link to interact with the visualizations.

## A.1.  Import Code

In [1]:
# python imports
import pandas as pd
import altair as alt
import requests
import numpy as np

# our imports
import intervals
from intervals import importScore 
from intervals.classify import *
from intervals.classify import batch_classify
from intervals.main_objs import CorpusBase
import intervals.visualizations as viz
# import visualizations_demo as viz_demo

## B. nGram Heatmap

### B.1. `plot_ngrams_heatmap`

* First, we do the normal steps to get a ngrams dataframe.

In [2]:
root = "https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/"
prefix = "CRIM_Model_0008"
files = [""] 
postfix = ".mei"
corpus = CorpusBase([root + prefix + files[0] + postfix])
piece = corpus.scores[0]
mel = piece.melodic(kind='d', compound=False, unit=0)
mel_ngrams = piece.ngrams(df=mel, n=5)
mel_ngrams.head()
mel_ngrams_bm = piece.detailIndex(mel_ngrams)
mel_ngrams_bm.head()

Successfully imported https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Model_0008.mei


,,[Superius],Altus,Tenor,Bassus
Measure,Beat,,,,
1,3.0,"(4, 1, 2, 2, -3)",NaN,NaN,NaN
3,3.0,NaN,"(4, 1, 2, 2, -3)",NaN,NaN
5,3.0,NaN,NaN,"(4, 1, 2, 2, -3)",NaN
7,3.0,NaN,NaN,NaN,"(4, 1, 2, 2, -3)"
8,4.0,"(-2, -2, -2, 2, -2)",NaN,NaN,NaN


### Plot Notes for All Voices

- Here we get all notes
- Fill Rests and NAS with "-"
- Plot all notes per voice

In [5]:
root = "https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/"
prefix = "CRIM_Model_0008"
files = [""] 
postfix = ".mei"
corpus = CorpusBase([root + prefix + files[0] + postfix])
piece = corpus.scores[0]
notes = piece.notes()
notes.replace("Rest", np.nan, inplace=True)


Previously imported piece detected.


In [6]:
viz.plot_ngrams_heatmap(notes, selected_patterns=[], voices=[])

alt.VConcatChart(...)

### B.2 Read Documentation

* Read the documentation with `help(viz_demo.plot_ngrams_heatmap)`

In [4]:
help(viz.plot_ngrams_heatmap)

Help on function plot_ngrams_heatmap in module intervals.visualizations:

plot_ngrams_heatmap(ngrams_df, ngrams_duration=None, selected_patterns=[], voices=[], heatmap_width=800, heatmap_height=300)
    Plot a heatmap for crim-intervals getNgram's output.
    :param ngrams_df: crim-intervals getNgram's output
    :param ngrams_duration: if not None, rely on durations in the
    df to calculate the durations of the ngrams.
    :param selected_patterns: list of specific patterns the users want (optional)
    :param voices: list of specific voices the users want (optional)
    :param heatmap_width: the width of the final heatmap (optional)
    :param heatmap_height: the height of the final heatmap (optional)
    :return: a bar chart that displays the different patterns and their counts,
    and a heatmap with the start offsets of chosen voices / patterns



* Note that in the plot tool below each **selected pattern** must be **strings assembled as a tuple, then all presented as list**:

> `selected_patterns=[('1', '2', '-2', '-2', '-2')]`

In [32]:

# note that each selected pattern must be strings assembled as a tuple, then all presented as list
# Thus: selected_patterns=[('1', '2', '-2', '-2', '-2')]
viz.plot_ngrams_heatmap(mel_ngrams,selected_patterns=[('1', '2', '-2', '-2', '-2')], voices=[])

alt.VConcatChart(...)

### B.3  Selecting nGrams

* We could only include certain nGrams in the heatmaps

* First, I collected the top patterns and turned these patterns into a list.

In [6]:
# count and get the 10 most popular patterns
mel_ngrams_top_patterns_df = mel_ngrams.stack().value_counts().to_frame().head(10)
# retrieve a list to generate heatmaps
mel_ngrams_top_patterns_list = mel_ngrams_top_patterns_df.index.to_list()
mel_ngrams_top_patterns_df

,0
"(1, 2, -2, -2, -2)",13
"(2, -2, -2, -2, -2)",11
"(1, 1, 2, -2, -2)",9
"(2, -3, 2, -2, -2)",8
"(-3, 2, -2, -2, -2)",8
"(-2, -2, -2, 2, 2)",7
"(2, -3, 2, -3, 2)",7
"(2, 2, 2, -3, 2)",6
"(2, 2, -3, 2, 2)",6
"(2, 2, 1, 2, -2)",6


In [7]:
mel_ngrams_top_patterns_list 

[('1', '2', '-2', '-2', '-2'),
 ('2', '-2', '-2', '-2', '-2'),
 ('1', '1', '2', '-2', '-2'),
 ('2', '-3', '2', '-2', '-2'),
 ('-3', '2', '-2', '-2', '-2'),
 ('-2', '-2', '-2', '2', '2'),
 ('2', '-3', '2', '-3', '2'),
 ('2', '2', '2', '-3', '2'),
 ('2', '2', '-3', '2', '2'),
 ('2', '2', '1', '2', '-2')]

* We would pass the list of patterns into the `patterns` parametter.

In [8]:
viz.plot_ngrams_heatmap(mel_ngrams, selected_patterns=mel_ngrams_top_patterns_list)

alt.VConcatChart(...)

### B.4 Check Locations of Individual nGram

#### **Cross-checking output from the heatmap with the ngrams dataframe**

* `'4, 1, 2, 2, -3'` is the opening melody of **Model_0008**.  Let's view its start points in ngram and in the heatmap's dataframe to make sure that it has been calculated correctly
* Note that in the plot tool below each **selected pattern** must be **strings assembled as a tuple, then all presented as list** Thus: `selected_patterns=[('1', '2', '-2', '-2', '-2')]`

In [33]:
mel_ngrams[mel_ngrams ==  '4, 1, 2, 2, -3'].stack()

Series([], dtype: object)

In [35]:
# note that each selected pattern must be strings assembled as a tuple, then all presented as list
# Thus: selected_patterns=[('1', '2', '-2', '-2', '-2')]

viz.plot_ngrams_heatmap(mel_ngrams, selected_patterns=[('1', '2', '-2', '-2', '-2')])
# viz_demo.plot_ngrams_heatmap(mel_ngrams, selected_patterns=mel_ngrams_top_patterns_list, voices=[])

alt.VConcatChart(...)

### B.5  Plotting nGrams with Durational Spans

* `plot_ngrams_heatmap` with Durational Spans 

* How could we create a heatmap for ngrams in which the ngrams' durations are displayed?

* Let's look at the method of interest, `model.getDuration()`'s documentation.

In [11]:
help(piece.durations)

Help on method durations in module intervals.main_objs:

durations(df=None, n=1, mask_df=None) method of intervals.main_objs.ImportedPiece instance
    If no arguments are passed, return a `pandas.DataFrame` of floats giving
    the duration of notes and rests in each part where 1 = quarternote,
    1.5 = a dotted quarter, 4 = a whole note, etc. If a df is passed, then
    return a df of the same shape giving the duration of each of the cells
    of this df. This is useful if you want to know what the durations of
    something other than single notes and rests, such as the durations of
    intervals. E.g.:
    
    har = importedPiece.harmonic()
    harDur = importedPiece.durations(df=har)
    
    The `n` parameter should be an integer greater than zero, or -1. When
    n is a positive integer, it groups together a sliding window of n
    consecutive non-NaN cells in each column. If you pass a df, it will sum
    the durations 'Rest' and non-Rest cell, provided they are in the same
 

Because we created `mel_ngrams` with the code above:

```
mel = model.getMelodic(kind='d', compound=False, unit=0)
mel_ngrams = model.getNgrams(df=mel, n=5)
mel_ngrams.head()
```

We would use `df=mel`, `n=5`, and `mask_df=mel_ngrams` for `model.getDuration()`.

In [12]:
mel_ngrams_duration = piece.durations(df=mel, n=4, mask_df=mel_ngrams)

In [37]:
viz.plot_ngrams_heatmap(mel_ngrams, mel_ngrams_duration, selected_patterns=[('1', '2', '-2', '-2', '-2')], voices=[])
# viz.plot_ngrams_heatmap(mel_ngrams, mel_ngrams_duration, selected_patterns=mel_ngrams_top_patterns_list, voices=[])

alt.VConcatChart(...)

### B.6 Plotting Close Matches

* `plot_close_match_heatmap`

* Compute the **Levenshtein distance** between the two ngrams and display them on a heatmap.

* You could real more about Levenshtein distance on [Wikipedia](https://en.wikipedia.org/wiki/Levenshtein_distance#:~:text=Informally%2C%20the%20Levenshtein%20distance%20between,considered%20this%20distance%20in%201965.).


In [15]:
help(viz.plot_close_match_heatmap)

Help on function plot_close_match_heatmap in module intervals.visualizations:

plot_close_match_heatmap(ngrams_df, key_pattern, ngrams_duration=None, selected_patterns=[], voices=[], heatmap_width=800, heatmap_height=300)
    Plot how closely the other vectors match a selected vector.
    Uses the Levenshtein distance.
    :param ngrams_df: crim-intervals getNgram's output
    :param key_pattern: a pattern the users selected to compare other patterns with (tuple of floats)
    :param selected_pattern: the specific other vectors the users selected
    :param ngrams_duration: if None, simply output the offsets. If the users input a
    list of durations, caculate the end by adding durations with offsets and
    display the end on the heatmap accordingly.
    :param selected_patterns: list of specific patterns the users want (optional)
    :param voices: list of specific voices the users want (optional)
    :param heatmap_width: the width of the final heatmap (optional)
    :param heatmap

In [16]:
viz.plot_close_match_heatmap(mel_ngrams, '4, 1, 2, 2, -3', selected_patterns=[], voices=[])

alt.Chart(...)

**Additional Clarification**:
    
- As we change the cutoff with the slider, the ngrams with scores lower than the cutoff would be hidden away.
- Levenshtein distance takes into account how many changes we need to make to turn a pattern into another. For example, pattern '2, 2, 1, 2' and pattern '2, 2, 1, 3' would have the same levenshtein score 25 when compared to pattern '2, 2, 1, 1' because we need to make one change at the fourth character of the two strings to turn it into '2, 2, 1, 1'

In [38]:
# use the durations calculated from above
viz.plot_close_match_heatmap(mel_ngrams,'4, 1, 2, 2, -3', mel_ngrams_duration, selected_patterns=[], voices=[])

alt.Chart(...)

In [21]:
distances = piece.distance(mel_ngrams)

dist_stack = distances.stack().to_frame()


    # filter distances to threshold.  <2 is good
filtered_dist_stack = dist_stack[dist_stack[0] < 2]
filtered_dist = filtered_dist_stack.reset_index()
filtered_dist.rename(columns =  {'level_0':"source", 'level_1':'match'}, inplace = True)
filtered_dist

,source,match,0
0,"(4, 1, 2, 2, -3)","(4, 1, 2, 2, -3)",0
1,"(-2, -2, -2, 2, -2)","(-2, -2, -2, 2, -2)",0
2,"(-2, -2, -2, 2, -2)","(-2, -2, -2, 3, -2)",1
3,"(-2, -2, -2, 2, -2)","(-2, -2, -2, 2, -3)",1
4,"(-2, -2, 2, -2, 4)","(-2, -2, 2, -2, 4)",0
...,...,...,...
542,"(-2, 2, 2, -2, 1)","(-2, 2, 3, -2, 1)",1
543,"(-2, 2, 2, -2, 1)","(-2, 2, 2, -2, 2)",1
544,"(-2, 2, 2, -2, 1)","(-2, 2, 2, -2, 1)",0
545,"(-4, 2, -2, -5, 1)","(-4, 2, -2, -5, 1)",0


## C. Heatmaps of CRIM Observation and Relationship Data

### C.1 Interactive Map of Relationships Type, Analyst, and  Location

* The `plot_comparison_heatmap` method can be used to plot a relationship/observation json.



In [19]:
data_relationships = requests.get('http://crimproject.org/data/relationships/').json()
df_relationships = pd.json_normalize(data_relationships)



In [3]:
# use this to filter for one piece
df_relationships_test = df_relationships[df_relationships['model_observation.piece.piece_id'] == 'CRIM_Model_0008'].copy()

**Plotting the same heatmap for other models/masses**:

You could refer to the list of available models/masses below and replace 'CRIM_Model_0008' with the piece of interest to plot the heatmap for that piece.

**List of Pieces** in the CRIM data set

In [4]:
df_relationships['model_observation.piece.piece_id'].unique()

array(['CRIM_Model_0011', 'CRIM_Model_0018', 'CRIM_Model_0017',
       'CRIM_Mass_0015_2', 'CRIM_Model_0001', 'CRIM_Model_0020',
       'CRIM_Model_0009', 'CRIM_Model_0019', 'CRIM_Model_0010',
       'CRIM_Model_0022', 'CRIM_Model_0002', 'CRIM_Model_0016',
       'CRIM_Model_0024', 'CRIM_Model_0021', 'CRIM_Mass_0018_1',
       'CRIM_Model_0008', 'CRIM_Mass_0018_3', 'CRIM_Model_0012',
       'CRIM_Model_0023', 'CRIM_Model_0015', 'CRIM_Model_0013',
       'CRIM_Mass_0005_1', 'CRIM_Mass_0016_2', 'CRIM_Mass_0020_3',
       'CRIM_Model_0006', 'CRIM_Model_0025', 'CRIM_Mass_0001_1',
       'CRIM_Mass_0001_2', 'CRIM_Mass_0033_1', 'CRIM_Model_0033',
       'CRIM_Model_0046', 'CRIM_Model_0050', 'CRIM_Model_0038',
       'CRIM_Model_0036'], dtype=object)

**Sample of the Relationships** derived from CRIM JSON

In [22]:
df_relationships_test.head()

,url,id,relationship_type,musical_type,remarks,created,updated,curated,observer.url,observer.name,...,details.old counter subject transposed,details.whole passage metrically shifted,details.double or invertible counterpoint,details.old counter subject shifted metrically,details.transposition,details.metrically shifted,details.systematic diminution,details.systematic augmentation,details,observer
869,https://crimproject.org/data/relationships/882/,882,mechanical transformation,imitative duo,,2017-07-25T19:32:02-04:00,2022-02-03T18:02:05.161870-05:00,True,https://crimproject.org/data/people/CRIM_Perso...,Stephanie Schlagel,...,NaN,NaN,False,NaN,transposed,True,None,None,NaN,NaN
870,https://crimproject.org/data/relationships/883/,883,non-mechanical transformation,"periodic entry, fuga",Last entry of four-voice PEn shifted and trans...,2017-07-25T19:32:02-04:00,2022-02-03T18:02:05.171264-05:00,True,https://crimproject.org/data/people/CRIM_Perso...,Stephanie Schlagel,...,True,None,None,True,NaN,NaN,NaN,NaN,NaN,NaN
969,https://crimproject.org/data/relationships/982/,982,new material,contrapuntal duo,,2017-07-25T23:31:47-04:00,2022-02-03T18:02:06.155103-05:00,True,https://crimproject.org/data/people/CRIM_Perso...,Stephanie Schlagel,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
970,https://crimproject.org/data/relationships/983/,983,non-mechanical transformation,"soggetto, non-imitative duo",S soggetto has been joined to a new CS and pre...,2017-07-25T23:31:47-04:00,2022-02-03T18:02:06.164545-05:00,True,https://crimproject.org/data/people/CRIM_Perso...,Stephanie Schlagel,...,False,None,None,False,NaN,NaN,NaN,NaN,NaN,NaN
971,https://crimproject.org/data/relationships/984/,984,non-mechanical transformation,fuga,A has been altered,2017-07-25T23:31:47-04:00,2022-02-03T18:02:06.174290-05:00,True,https://crimproject.org/data/people/CRIM_Perso...,Stephanie Schlagel,...,False,None,None,False,NaN,NaN,NaN,NaN,NaN,NaN


**A Single Relationship in JSON format**

In [23]:
df_relationships_test.iloc[0, :]

url                                                 https://crimproject.org/data/relationships/882/
id                                                                                              882
relationship_type                                                         mechanical transformation
musical_type                                                                          imitative duo
remarks                                                                                            
created                                                                   2017-07-25T19:32:02-04:00
updated                                                            2022-02-03T18:02:05.161870-05:00
curated                                                                                        True
observer.url                                      https://crimproject.org/data/people/CRIM_Perso...
observer.name                                                                    Stephanie Schlagel


**Documentation for Relationship/Observation Heatmap**

In [5]:
help(viz.plot_comparison_heatmap)

Help on function plot_comparison_heatmap in module intervals.visualizations:

plot_comparison_heatmap(df, ema_col, main_category='musical_type', other_category='observer.name', option=1, heat_map_width=800, heat_map_height=300)
    This method plots a chart for relationships/observations dataframe retrieved from their
    corresponding json files. This chart has two bar charts displaying the count of variables
    the users selected, and a heatmap displaying the locations of the relationship.
    :param df: relationships or observations dataframe
    :param ema_col: name of the ema column
    :param main_category: name of the main category for the first bar chart.
    The chart would be colored accordingly (default='musical_type').
    :param other_category: name of the other category for the zeroth bar chart.
    (default='observer.name')
    :param heat_map_width: the width of the final heatmap (default=800)
    :param heat_map_height: the height of the final heatmap (default =300)
 

**View Heatmap of Relationships for a Single Piece**

In [6]:
# use df_relationship_test for one piece as defined above, or df_relationships for all of them

viz.plot_comparison_heatmap(df_relationships_test, 'model_observation.ema', main_category='relationship_type', other_category='observer.name', heat_map_width=800,
                              heat_map_height=300)

alt.VConcatChart(...)

### C2. Interactive Map of Observations Type, Analyst, and  Location

We can **command+click on an color bars** to open a new tab that displays the given **observation**

In [36]:
data_observations = requests.get('http://crimproject.org/data/observations/').json()
df_observations = pd.json_normalize(data_observations)


In [38]:
df_observations.iloc[0, :]

url                                       https://crimproject.org/data/observations/1/
id                                                                                   1
ema                                  1-6/1,1,1-2,1-2,2,2/@1,@1-3,@1-3+@1,@1-3+@1-3,...
musical_type                                                                      fuga
remarks                                                           Makes up a longer ID
created                                                      2017-06-19T19:22:54-04:00
updated                                               2022-02-03T18:01:55.831239-05:00
curated                                                                           True
observer.url                         https://crimproject.org/data/people/CRIM_Perso...
observer.name                                                               Ian Lorenz
piece.url                            https://crimproject.org/data/pieces/CRIM_Model...
piece.piece_id                             

In [27]:
# Here you can select a particular piece

df_observations_test = df_observations[df_observations['piece.piece_id'] == 'CRIM_Model_0008'].copy()

In [28]:
# use df_observations_test for one piece, or df_observations for all of them

viz.plot_comparison_heatmap(df_observations_test, 'ema', main_category='musical_type', other_category='observer.name', heat_map_width=800,
                              heat_map_height=300)

alt.VConcatChart(...)

## D.  CRIM Metadata as Networks 

* uses `create_comparisons_networks_and_interactive_df`

We can create a some network visualizations and a df to interact with patterns.

Work with both time interval and melodic interval!

**Important**:
- The plot will be saved in a html file in the folder you choose.
- The plot might not be displayed if it's not saved in the same folder as the Jupyter Notebook, but you should be able to find it and open it in your computer if necessary.
- The `segments` column in the interactive df shows the range of the measures the observation was found, for example: `['5-9', '14-18']` means that the observation have been found from measure 5 to measure 9, and again from measure 14 to measure 18.



### D.1  Network of PEns by Melodic Intervals between Entries

The columns of interest from table containing observations fetched from the CRIM database would be `mt_pe_int` (`mt` = musical type, `pe` = periodic entries, `int`= intervals) and we would choose `interval_type` to be `melodic intervals`. The table's corresponding ema column is `ema`.

In [39]:
pen_networks, pen_widget = viz.create_comparisons_networks_and_interactive_df(df_observations, 'musical_type', 'melodic', 'ema')

interactive(children=(Text(value='Input search pattern', description='search_pattern_starts_with'), Output()),…

**See all the available starting intervals**:

- the 'all' key means that this networks contains all of the pens.
- If you select a network with specific starting intervals (e.g: 8+), it only contains pens that start with 8+.

In [ ]:
pen_networks

In [ ]:
# show all the patterns mapped onto one network
pen_networks['all'].show('all_pen.html')


In [ ]:
# show only one family of patterns mapped onto one network
pen_networks['8+'].show('networks/8+_pen.html')

### D.2  Network of Fugas by Melodic Intervals between Entries

The columns of interest from table containing observations fetched from the CRIM database would be `mt_fg_int` (mt= musical type, fg= Fuga, int=intervals) and we would choose `interval_type` to be `melodic`. The table's corresponding ema column is `ema`.

In [ ]:
fuga_networks, fuga_widget = viz.create_comparisons_networks_and_interactive_df(df_observations, 'mt_fg_int', 'melodic', 'ema')

In [ ]:
# we are interested in the patterns that start with 4-
# fuga_networks['4-'].show('networks/4-_fuga.html')
fuga_networks['4-'].show('4-_fuga.html')

In [ ]:
fuga_networks

### D.3. CRIM Relationships as Network of Pieces

* `plot_relationship_network` method

This method accepts a dataframe of CRIM relationships and generate a network of observations.

In the network:

- the nodes are the observations (labeled with the piece and measures they are in)
- the edges are the relationships. 
    - points from a model_observation to a derivative observation.
    - are weighted differently based on the type of the relationships.
    - are labeled with the relationship type
    - are colored based on the users' choice.

**WARNING**: Edge color isn't always a correct indicator of the relationship type.

The color of the edges is derived from the nodes; therefore, they would not be colored correctly when:

1. color='derivative'

model1 --(quotation)--> model2

model3 --(new material)--> model2

because of the first relationship, model2 and the two edges pointing to it would be colored with quotation's color.

2. color='model'

model1 --(quotation)--> model2

model1 --(new material)--> model3

because of the first relationship, model1 and the two out edges from model1 would be colored with quotation's color

In this case, we should pay attention to the edges' thickness (different weights mean different relationship types) or hover over the edges to see the relationship type labels.

**Seeing the big picture with the color modes being `model` or `derivative`**

We have two options for the `color` parameter. 

- `derivative` (default): the edges and nodes of derivative observations will be colored according to the relationships. 
- `model` (default): the edges and nodes of model observations will be colored according to the relationships. 

**Here I use df_relationships.head(300) instead of df_relationships because plotting all of the relationships usually takes really long**.

#### color=`derivative`

In [31]:
nt = viz.plot_relationship_network(df_relationships.head(300), color='derivative')
nt.show('small_relationships_network_derivative.html')

AttributeError: module 'intervals.visualizations' has no attribute 'plot_relationship_network'

#### color=`model`

In [30]:
nt2 = viz.plot_relationship_network(df_relationships.head(300), color='model')
nt2.show('small_relationships_network_model.html')

AttributeError: module 'intervals.visualizations' has no attribute 'plot_relationship_network'

### D.4  Networks with Specific Pieces (In Progress)

The two plots are too cluttered so we can't see the relationship_type between specific measures inside pieces. 

Selecting **specific models** with `selected_model_ids` and **specific derivatives** with`selected_derivative_ids` 

The first example shows all the movements of Févin's Missa Ave Maria as a network.  This reveals which parts of the model have been used where, since 'centers' on the map represent particular measure ranges from the motet.



In [32]:
# nt3 = viz_demo.plot_relationship_network(df_relationships, selected_derivative_ids=['CRIM_Mass_0017_3', 'CRIM_Mass_0008_4',
#        'CRIM_Mass_0003_5', 'CRIM_Mass_0014_4', 'CRIM_Mass_0018_1',
#        'CRIM_Mass_0018_2', 'CRIM_Mass_0005_5'], color='derivative')
# nt3.show('selected_derivatives.html')

nt3 = viz.plot_relationship_network(df_relationships, selected_derivative_ids=[
       'CRIM_Mass_0005_3', 'CRIM_Mass_0005_2', 'CRIM_Mass_0005_1',
       'CRIM_Mass_0005_4', 'CRIM_Mass_0005_5'], color='derivative')
nt3.show('selected_derivatives.html')

AttributeError: module 'intervals.visualizations' has no attribute 'plot_relationship_network'

**Network for One Model**

* Here is the same network mapped from the perspective of a model

In [33]:
nt4 = viz.plot_relationship_network(df_relationships, selected_model_ids = ['CRIM_Model_0008'], color='derivative')
nt4.show('selected_model.html')

AttributeError: module 'intervals.visualizations' has no attribute 'plot_relationship_network'

#### Plotting pieces that are relevant to a list of interested pieces with `selected_members`
Having some of interest pieces, we want to plot all pieces that happen to connect to these pieces somehow.

In [34]:
nt5 = viz.plot_relationship_network(df_relationships, selected_model_ids = ['CRIM_Model_0008', 'CRIM_Model_0016'], selected_families=['CRIM_Mass_0017_3', 'CRIM_Mass_0008_4',
       'CRIM_Mass_0003_5', 'CRIM_Mass_0014_4', 'CRIM_Mass_0018_1',
       'CRIM_Mass_0018_2', 'CRIM_Mass_0005_5'], color='derivative')
nt5.show('selected_model_families.html')

AttributeError: module 'intervals.visualizations' has no attribute 'plot_relationship_network'